# <center> Create Yearly Collection KPI data </center>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"D:\MyStuff\Anne\WeeklyKPI\New folder\Yearly_monthend_data.csv")
print("The shape of the DataFrame is: t", df.shape)

The shape of the DataFrame is: t (9, 17)


In [3]:
# Let us check the data
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,bucket1contacted,bucketcallcontacted,bucket1normalized_greg,bucket1normalized_denominator,bucket2normalized_denominator,bucket2normalized_greg,ptp_received,kept_ptp,bucket1ptppmt,bucket2ptppmt,FTE,total_accounts,active_works
0,01-01-2023,31-01-2023,2078,2458,1098,3485,1974,3613,2225,244,1917,327,764647.87,126419.52,23,18940,6608
1,01-02-2023,28-02-2023,2034,2347,842,2539,1804,3640,2061,185,1747,286,653817.86,168324.40,22,21398,6716
2,01-03-2023,31-03-2023,2310,2867,1237,3934,2742,4024,1984,239,1685,505,1197285.16,200716.69,21,23445,6921
3,01-04-2023,30-04-2023,2487,3565,638,2705,2439,4061,2099,254,1774,369,969378.89,113452.10,20,24623,6327
4,01-05-2023,31-05-2023,3080,4618,718,3128,3027,4424,2217,359,1996,654,1758382.65,168261.05,20,22604,6550
5,01-06-2023,30-06-2023,2776,3939,491,3152,2783,4304,2385,403,1967,443,1054469.60,184900.23,20,25791,6478
6,01-07-2023,31-07-2023,3175,4540,537,3589,3212,4890,2669,498,2213,596,1573514.32,149953.60,20,25434,6904
7,01-08-2023,31-08-2023,4215,5407,572,4096,3683,5428,2993,575,2460,721,1767505.16,227259.71,20,26972,7364
8,01-09-2023,30-09-2023,3772,5280,617,4661,3757,5870,3290,642,2575,588,1371037.92,215522.64,27,29436,8203


In [4]:
# Import library to connect to Google Cloud SQL


from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")



Make sure to update the date before executing this query

In [5]:
sq = """
WITH
  dt AS (
  SELECT
    DATE('2023-10-01') WkstartDate,
    DATE('2023-10-30') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
)
,
wc1 as 
(
SELECT
  dt.WkstartDate,
  dt.WkendDate,
  Genesys_call_id,
  callDatetime,
  COALESCE(IS_APC, 0) APC,
  COALESCE(IS_RPC, 0) RPC,
  COALESCE(IS_PTP,0) PTP,
  RIGHT(mobileNumber, 10) mobileNumber,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult,
  ROW_NUMBER() OVER(PARTITION BY Genesys_call_id ORDER BY callDatetime DESC, COALESCE(IS_APC, 0), COALESCE(IS_RPC, 0), COALESCE(IS_PTP, 0))rnk
FROM
  `risk_credit_mis.call_attempt_history_gensys` g,  dt
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm
ON
  UPPER(cm.Campaignname) = UPPER(g.campaignName)
INNER JOIN disb ON  SUBSTR(disb.mobileNo, -10) = SUBSTR(CAST(g.mobileNumber AS string), -10)
WHERE
  cm.reference = 'WC'
  and disb.rnk = 1
  and date(callDatetime) between  dt.WkstartDate and dt.WkendDate
)
, 
wc as 
(select wc1.WkstartDate, wc1.WkendDate, Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
case when APC = 1 then mobileNumber end apc_m,
case when RPC = 1 then mobileNumber end rpc_m,
case when PTP = 1 then mobileNumber end ptp_m,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
from wc1 
where rnk = 1
)
,
wcbase as 
( 
select wc.WkstartDate, wc.WkendDate,
 Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
apc_m,
rpc_m,
ptp_m
from wc
)
, 
cd AS (
  SELECT
    customer_id,
    new_mobile_number,
    ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY change_date)rnk
  FROM
    `risk_credit_mis.customer_contact_details`
  WHERE
    ACTIVE = 'Y')
  ,
  buck1 as 
  (select distinct dt.WkstartDate, dt.WkendDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD, lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber
  , row_number() over(partition by lbfrc.loanAccountNumber order by lbfrc.bucketDate desc) buck1rnk
  from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
  left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
  left join cd on cd.customer_id = lmt.customerId
  where date(bucketDate) between  date(dt.WkstartDate) and  date(dt.WkendDate)
  and coalesce(lbfrc.Max_current_DPD, 0) between 1 and 30 
  and date(lbfrc.firstDueDate) <= date(current_date())
  and lbfrc.loanStatus = 'In Arrears'
  and cd.rnk = 1
  )
    ,
buck1cr1 as 
(select distinct
 dt.WkstartDate,
  dt.WkendDate,
  g.Genesys_call_id,
  g.callDatetime,
  COALESCE(g.IS_APC, 0) APC,
  COALESCE(g.IS_RPC, 0) RPC,
  COALESCE(g.IS_PTP,0) PTP,
  RIGHT(g.mobileNumber, 10) mobileNumber,
  buck1.newmobilenumber mobilefrombuckettable,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult, 
  buck1.loanAccountNumber,
  buck1.DPD,
  case when g.IS_APC = 1 then g.mobileNumber end apc_m,
  case when g.IS_RPC = 1 then g.mobileNumber end rpc_m,
  case when g.IS_PTP = 1 then mobileNumber end ptp_m,
  ROW_NUMBER() OVER(PARTITION BY g.Genesys_call_id ORDER BY g.callDatetime DESC, COALESCE(g.IS_APC, 0), COALESCE(g.IS_RPC, 0), COALESCE(g.IS_PTP, 0))rnk 
from  `risk_credit_mis.call_attempt_history_gensys` g,  dt
inner join buck1 on buck1.newmobilenumber = right(g.mobileNumber,10)
LEFT JOIN  `dap_ds_poweruser_playground.campaignmaster` cm ON  UPPER(cm.Campaignname) = UPPER(g.campaignName)
where cm.Reference = 'COLL' and cm.Subcategory = 'SOFT'
AND DATE(g.callDatetime) BETWEEN DATE(dt.WkstartDate) AND DATE(dt.WkendDate)
and buck1.buck1rnk = 1
)
,
buck1cr1base as 
(select buck1cr1.WkstartDate, buck1cr1.WkendDate, buck1cr1.Genesys_call_id, buck1cr1.callDatetime
, buck1cr1.mobileNumber
, buck1cr1.mobilefrombuckettable
, buck1cr1.campaignName
, buck1cr1.agentGroup
, buck1cr1.agentFullName
, buck1cr1.employeeId
, buck1cr1.connected
, buck1cr1.notConnected
, buck1cr1.callResult
, buck1cr1.loanAccountNumber
, buck1cr1.apc_m 
, buck1cr1.rpc_m
, buck1cr1.ptp_m 
, buck1cr1.DPD
, buck1cr1.rnk 
 from buck1cr1 where rnk = 1
)
,
buck1end as 
(select distinct dt.WkstartDate, dt.WkendDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD, lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber
from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
left join cd on cd.customer_id = lmt.customerId
where date(bucketDate) = date(dt.WkendDate)
and date(lbfrc.firstDueDate) <= date(current_date())
and cd.rnk = 1
)
, 
ptpstatus as 
(select max(dt.WkstartDate)WkstartDate, max(dt.WkendDate)WkendDate, cps.Loan_account_no, sum(case when coalesce(cps.PTP_Status, 0) = 1 then 1 else 0 end) ptpstatus 
from `risk_credit_mis.collection_ptp_status`  cps, dt where coalesce(PTP_Status, 0) = 1
and date(contact_date) between dt.WkstartDate and dt.WkendDate
group by cps.Loan_account_no
),
payments as (select loanAccountNumber from `risk_credit_mis.loan_installments_table`, dt where date(lastPaymentDate) between dt.WkstartDate and dt.WkendDate) 
,
buck1normalized as 
(select buck1.WkstartDate, buck1.WkendDate, buck1.loanAccountNumber , buck1.DPD weekstartDPD, buck1end.DPD weekendDPD
, case when buck1.loanAccountNumber in (select distinct Loan_account_no from `risk_credit_mis.collection_ptp_status` where PTP_Due_Date is not null) then buck1.newmobilenumber end ptp_received
, case when buck1.loanAccountNumber in (select distinct Loan_account_no from ptpstatus) then buck1.newmobilenumber end Kept_ptp
, case when coalesce(buck1.DPD, 0) between 1 and 30 and coalesce(buck1end.DPD, 0) = 0 and buck1.loanAccountNumber in (select distinct Loan_account_no from ptpstatus) then buck1.newmobilenumber end Normalized 
, case when coalesce(buck1.DPD, 0) between 1 and 30 and coalesce(buck1end.DPD, 0) = 0 and buck1.loanAccountNumber not in (select distinct Loan_account_no from ptpstatus) then buck1.newmobilenumber end Normalizedwithoutptp 
, case when coalesce(buck1.DPD, 0) between 1 and 30 and coalesce(buck1end.DPD, 0) <= 30 and buck1.loanAccountNumber in (select loanAccountNumber from payments) then buck1.newmobilenumber end Normalized_greg 
, case when coalesce(buck1.DPD, 0) between 1 and 30 then buck1.newmobilenumber end Normalized_greg_denominator
, case when coalesce(buck1.DPD, 0) between 1 and 30 and coalesce(buck1end.DPD, 0) <= 30 and buck1.loanAccountNumber not in (select distinct Loan_account_no from ptpstatus) then buck1.newmobilenumber end Normalizedwithoutptp_greg
from buck1
left join buck1end on buck1.loanAccountNumber =  buck1end.loanAccountNumber
)
,
buck2 as 
(select dt.WkstartDate, dt.WkendDate, lbfrc.loanAccountNumber, lbfrc.loanStatus, lbfrc.bucketDate, lbfrc.Max_current_DPD as DPD, lmt.customerId, right(cd.new_mobile_number, 10) newmobilenumber
, row_number() over(partition by lbfrc.loanAccountNumber order by lbfrc.bucketDate desc) buck2rnk
from `risk_credit_mis.loan_bucket_flow_report_core` lbfrc, dt 
left join `risk_credit_mis.loan_master_table` lmt on lmt.loanAccountNumber = lbfrc.loanAccountNumber
left join cd on cd.customer_id = lmt.customerId
where date(bucketDate)  between  date(dt.WkstartDate) and  date(dt.WkendDate)
and coalesce(lbfrc.Max_current_DPD, 0) between 31 and 60 
and date(lbfrc.firstDueDate) <= date(current_date())
and lbfrc.loanStatus = 'In Arrears'
and cd.rnk = 1
)
,
buck2cr1 as 
(select   dt.WkstartDate,
  dt.WkendDate,
  g.Genesys_call_id,
  g.callDatetime,
  COALESCE(g.IS_APC, 0) APC,
  COALESCE(g.IS_RPC, 0) RPC,
  COALESCE(g.IS_PTP,0) PTP,
  RIGHT(g.mobileNumber, 10) mobileNumber,
  buck2.newmobilenumber mobilefrombuckettable,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult, 
  buck2.loanAccountNumber,
  buck2.DPD,
  case when g.IS_APC = 1 then g.mobileNumber end apc_m,
  case when g.IS_RPC = 1 then g.mobileNumber end rpc_m,
  case when g.IS_PTP = 1 then mobileNumber end ptp_m,
  ROW_NUMBER() OVER(PARTITION BY g.Genesys_call_id ORDER BY g.callDatetime DESC, COALESCE(g.IS_APC, 0), COALESCE(g.IS_RPC, 0), COALESCE(g.IS_PTP, 0))rnk 
from  `risk_credit_mis.call_attempt_history_gensys` g,  dt
inner join buck2 on buck2.newmobilenumber = right(g.mobileNumber,10)
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm ON  UPPER(cm.Campaignname) = UPPER(g.campaignName)
where cm.Reference = 'COLL' 
AND DATE(g.callDatetime) BETWEEN DATE(dt.WkstartDate) AND DATE(dt.WkendDate)
and buck2.buck2rnk = 1
)
,buck2cr1base as 
(select buck2cr1.WkstartDate, buck2cr1.WkendDate, buck2cr1.Genesys_call_id, buck2cr1.callDatetime
, buck2cr1.mobileNumber
, buck2cr1.mobilefrombuckettable
, buck2cr1.campaignName
, buck2cr1.agentGroup
, buck2cr1.agentFullName
, buck2cr1.employeeId
, buck2cr1.connected
, buck2cr1.notConnected
, buck2cr1.callResult
, buck2cr1.loanAccountNumber
, buck2cr1.apc_m 
, buck2cr1.rpc_m
, buck2cr1.ptp_m 
, buck2cr1.DPD
, buck2cr1.rnk 
 from buck2cr1 where rnk = 1
)
,
buck2normalized as 
(select buck2.WkstartDate, buck2.WkendDate, buck2.loanAccountNumber , buck2.DPD weekstartDPD, buck1end.DPD weekendDPD
, case when buck2.loanAccountNumber in (select distinct Loan_account_no from `risk_credit_mis.collection_ptp_status` where PTP_Due_Date is not null) then buck2.newmobilenumber end ptp_received
, case when buck2.loanAccountNumber in (select distinct Loan_account_no from ptpstatus) then buck2.newmobilenumber end Kept_ptp
, case when coalesce(buck2.DPD, 0) between 31 and 60 and coalesce(buck1end.DPD, 0) = 0 and buck2.loanAccountNumber in (select distinct Loan_account_no from ptpstatus) then buck2.newmobilenumber end Normalized 
, case when coalesce(buck2.DPD, 0) between 31 and 60 and coalesce(buck1end.DPD, 0) = 0 and buck2.loanAccountNumber not in (select distinct Loan_account_no from ptpstatus) then buck2.newmobilenumber end Normalizedwithoutptp 
, case when coalesce(buck2.DPD, 0) between 31 and 60 then buck2.newmobilenumber end Normalized_greg_denominator 
, case when coalesce(buck2.DPD, 0) between 31 and 60 and coalesce(buck1end.DPD, 0) <= 60 and buck2.loanAccountNumber in (select loanAccountNumber from payments) then buck2.newmobilenumber end Normalized_greg 
, case when coalesce(buck2.DPD, 0) between 31 and 60 and coalesce(buck1end.DPD, 0) <= 60 and buck2.loanAccountNumber not in (select distinct Loan_account_no from ptpstatus) then buck2.newmobilenumber end Normalizedwithoutptp_greg 
from buck2
left join buck1end on buck2.loanAccountNumber =  buck1end.loanAccountNumber
)
,
Ptp_payment as 
(select date_trunc(contact_date, day) contact_date, Loan_account_no, Phone_no, sum(case when coalesce(PTP_Status, 0) > 0 then PTP_Payment_amount else 0 end) ptppaymentamount 
from prj-prod-dataplatform.risk_credit_mis.collection_ptp_status group by date_trunc(contact_date, day), Loan_account_no, Phone_no
)
,
buck1ptpamt as 
(select distinct buck1normalized.WkstartDate,buck1normalized.WkendDate,loanAccountNumber
, case when Normalized_greg is not null then (select sum(ptppaymentamount) from Ptp_payment where Loan_account_no = buck1normalized.loanAccountNumber and date(contact_date) between dt.WkstartDate and dt.WkendDate) else 0 end ptpamount_normalized  
,(select sum(ptppaymentamount) from Ptp_payment where Loan_account_no = buck1normalized.loanAccountNumber and date(contact_date) between dt.WkstartDate and dt.WkendDate)  ptpamount  
from buck1normalized, dt)
,
buck2ptpamt as 
(select distinct buck2normalized.WkstartDate,buck2normalized.WkendDate,loanAccountNumber
, case when Normalized is not null then (select sum(ptppaymentamount) from Ptp_payment where Loan_account_no = buck2normalized.loanAccountNumber and date(contact_date) 
between dt.WkstartDate and dt.WkendDate) else 0 end ptpamount_normalized  
, (select sum(ptppaymentamount) from Ptp_payment where Loan_account_no = buck2normalized.loanAccountNumber and date(contact_date) 
between dt.WkstartDate and dt.WkendDate)  ptpamount 
from buck2normalized, dt),
fte as 
(select distinct agentFullName FROM
  `risk_credit_mis.call_attempt_history_gensys` g,
  dt
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm
ON
  UPPER(cm.Campaignname) = UPPER(g.campaignName)
WHERE
   DATE(g.callDatetime) BETWEEN DATE(dt.WkstartDate)  AND DATE(WkendDate)
   and cm.reference in ('REM', 'COLL', 'WC')),
LUM as 
(select distinct loanAccountNumber from `prj-prod-dataplatform.risk_credit_mis.loan_master_table`, dt where flagDisbursement = 1 and date_trunc(disbursementDateTime, day) < dt.WkstartDate
and loanPaidStatus not in ('Written Off','Completed', 'Settled')),
aw as 
(select bucketDate, loanAccountNumber from `risk_credit_mis.loan_bucket_flow_report_core`, dt where coalesce(Max_current_DPD ,0) between 1 and 179 and date(bucketDate) = dt.WkstartDate and loanStatus not in ('Written Off','Completed', 'Settled'))
select 
(select WkstartDate from dt) WkstartDate,
(select WkendDate from dt) WkendDate,
(select count(distinct apc_m) from wc) apc_wc_connected,
(select count(distinct mobileNumber) wccontactrate from wc) wccalled,
(select count(distinct apc_m) from buck1cr1base) bucket1contacted,
(select count(distinct mobilefrombuckettable) Bucket1callcontactrate from buck1cr1base where mobilefrombuckettable is not null)bucketcallcontacted,
(select count(distinct Normalized_greg) from buck1normalized) bucket1normalized_greg,
(select count(distinct Normalized_greg_denominator) from buck1normalized) bucket1normalized_denominator,
(select count(distinct Normalized_greg_denominator) from buck2normalized) bucket2normalized_denominator,
(select count(distinct Normalized_greg) from buck2normalized) bucket2normalized_greg,
(select count(distinct ptp_received) from buck1normalized)+(select count(distinct ptp_received) from buck2normalized) ptp_received,
(select count(distinct Kept_ptp) from buck1normalized) + (select count(distinct Kept_ptp) from buck2normalized) kept_ptp,
(select sum(ptpamount) from buck1ptpamt)bucket1ptppmt,
(select sum(ptpamount) from buck2ptpamt) bucket2ptppmt,
(select count(distinct agentFullName) from fte)FTE,
(select count(distinct loanAccountNumber) from LUM) total_accounts,
(select count(distinct loanAccountNumber) from aw) active_works
"""


In [6]:
df1 = client.query(sq).to_dataframe()

In [7]:
print("The shape of Data Frame for October 2023 data is: t", df1.shape)

The shape of Data Frame for October 2023 data is: t (1, 17)


In [8]:
df1

,WkstartDate,WkendDate,apc_wc_connected,wccalled,bucket1contacted,bucketcallcontacted,bucket1normalized_greg,bucket1normalized_denominator,bucket2normalized_denominator,bucket2normalized_greg,ptp_received,kept_ptp,bucket1ptppmt,bucket2ptppmt,FTE,total_accounts,active_works
0,2023-10-01,2023-10-30,3976,5667,1542,5592,3558,6306,3692,697,3020,832,1970487.79,319901.85,27,33079,9133


In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   WkstartDate                    1 non-null      dbdate 
 1   WkendDate                      1 non-null      dbdate 
 2   apc_wc_connected               1 non-null      Int64  
 3   wccalled                       1 non-null      Int64  
 4   bucket1contacted               1 non-null      Int64  
 5   bucketcallcontacted            1 non-null      Int64  
 6   bucket1normalized_greg         1 non-null      Int64  
 7   bucket1normalized_denominator  1 non-null      Int64  
 8   bucket2normalized_denominator  1 non-null      Int64  
 9   bucket2normalized_greg         1 non-null      Int64  
 10  ptp_received                   1 non-null      Int64  
 11  kept_ptp                       1 non-null      Int64  
 12  bucket1ptppmt                  1 non-null      float64

In [9]:
def convert_date_format(date_str):
    input_date = pd.to_datetime(date_str, format='%Y-%m-%d')
    return input_date.strftime('%d-%m-%Y')

In [10]:
# Apply the conversion function to the 'date_column'
df1['WkstartDate'] = df1['WkstartDate'].apply(convert_date_format)
df1['WkendDate'] = df1['WkendDate'].apply(convert_date_format)

In [11]:
df1.head()

,WkstartDate,WkendDate,apc_wc_connected,wccalled,bucket1contacted,bucketcallcontacted,bucket1normalized_greg,bucket1normalized_denominator,bucket2normalized_denominator,bucket2normalized_greg,ptp_received,kept_ptp,bucket1ptppmt,bucket2ptppmt,FTE,total_accounts,active_works
0,01-10-2023,30-10-2023,3976,5667,1542,5592,3558,6306,3692,697,3020,832,1970487.79,319901.85,27,33079,9133


In [12]:
df = df.append(df1)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 0
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   WkstartDate                    10 non-null     object 
 1   WkendDate                      10 non-null     object 
 2   apc_wc_connected               10 non-null     Int64  
 3   wccalled                       10 non-null     Int64  
 4   bucket1contacted               10 non-null     Int64  
 5   bucketcallcontacted            10 non-null     Int64  
 6   bucket1normalized_greg         10 non-null     Int64  
 7   bucket1normalized_denominator  10 non-null     Int64  
 8   bucket2normalized_denominator  10 non-null     Int64  
 9   bucket2normalized_greg         10 non-null     Int64  
 10  ptp_received                   10 non-null     Int64  
 11  kept_ptp                       10 non-null     Int64  
 12  bucket1ptppmt                  10 non-null     float6

In [14]:
int_columns = df.select_dtypes(include=['int64']).columns
df[int_columns] = df[int_columns].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 0
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   WkstartDate                    10 non-null     object 
 1   WkendDate                      10 non-null     object 
 2   apc_wc_connected               10 non-null     int32  
 3   wccalled                       10 non-null     int32  
 4   bucket1contacted               10 non-null     int32  
 5   bucketcallcontacted            10 non-null     int32  
 6   bucket1normalized_greg         10 non-null     int32  
 7   bucket1normalized_denominator  10 non-null     int32  
 8   bucket2normalized_denominator  10 non-null     int32  
 9   bucket2normalized_greg         10 non-null     int32  
 10  ptp_received                   10 non-null     int32  
 11  kept_ptp                       10 non-null     int32  
 12  bucket1ptppmt                  10 non-null     float6

In [15]:
df.head(20)

,WkstartDate,WkendDate,apc_wc_connected,wccalled,bucket1contacted,bucketcallcontacted,bucket1normalized_greg,bucket1normalized_denominator,bucket2normalized_denominator,bucket2normalized_greg,ptp_received,kept_ptp,bucket1ptppmt,bucket2ptppmt,FTE,total_accounts,active_works
0,01-01-2023,31-01-2023,2078,2458,1098,3485,1974,3613,2225,244,1917,327,764647.87,126419.52,23,18940,6608
1,01-02-2023,28-02-2023,2034,2347,842,2539,1804,3640,2061,185,1747,286,653817.86,168324.40,22,21398,6716
2,01-03-2023,31-03-2023,2310,2867,1237,3934,2742,4024,1984,239,1685,505,1197285.16,200716.69,21,23445,6921
3,01-04-2023,30-04-2023,2487,3565,638,2705,2439,4061,2099,254,1774,369,969378.89,113452.10,20,24623,6327
4,01-05-2023,31-05-2023,3080,4618,718,3128,3027,4424,2217,359,1996,654,1758382.65,168261.05,20,22604,6550
5,01-06-2023,30-06-2023,2776,3939,491,3152,2783,4304,2385,403,1967,443,1054469.60,184900.23,20,25791,6478
6,01-07-2023,31-07-2023,3175,4540,537,3589,3212,4890,2669,498,2213,596,1573514.32,149953.60,20,25434,6904
7,01-08-2023,31-08-2023,4215,5407,572,4096,3683,5428,2993,575,2460,721,1767505.16,227259.71,20,26972,7364
8,01-09-2023,30-09-2023,3772,5280,617,4661,3757,5870,3290,642,2575,588,1371037.92,215522.64,27,29436,8203
0,01-10-2023,30-10-2023,3976,5667,1542,5592,3558,6306,3692,697,3020,832,1970487.79,319901.85,27,33079,9133


Make sure to change the file name

In [16]:
df.to_csv(r"D:\MyStuff\Anne\WeeklyKPI\New folder\Yearly_monthend_data_October2023.csv")

In [17]:
# Convert DataFrame to pivot table with a calculated field

pivot_df = df.pivot_table( index=['WkstartDate', 'WkendDate'],values=['apc_wc_connected', 'wccalled','bucket1contacted', 'bucketcallcontacted'
                                                                      , 'bucket1normalized_greg', 'bucket1normalized_denominator'
                                                                      , 'bucket2normalized_greg', 'bucket2normalized_denominator'
                                                                      , 'kept_ptp', 'ptp_received'
                                                                      , 'bucket1ptppmt', 'bucket2ptppmt'
                                                                                                                                            ],
                          aggfunc={'apc_wc_connected': 'sum', 'wccalled': 'sum', 'bucket1contacted': 'sum', 'bucketcallcontacted':'sum'
                                   , 'bucket1normalized_greg':'sum', 'bucket1normalized_denominator':'sum'
                                   , 'bucket2normalized_greg':'sum', 'bucket2normalized_denominator':'sum'
                                   , 'kept_ptp':'sum', 'ptp_received':'sum'
                                   , 'bucket1ptppmt':'sum', 'bucket2ptppmt':'sum'
                                   
                                   },
                          margins=True, margins_name='Total', fill_value=0)

In [18]:
pivot_df

,,apc_wc_connected,bucket1contacted,bucket1normalized_denominator,bucket1normalized_greg,bucket1ptppmt,bucket2normalized_denominator,bucket2normalized_greg,bucket2ptppmt,bucketcallcontacted,kept_ptp,ptp_received,wccalled
WkstartDate,WkendDate,,,,,,,,,,,,
01-01-2023,31-01-2023,2078,1098,3613,1974,764647.87,2225,244,126419.52,3485,327,1917,2458
01-02-2023,28-02-2023,2034,842,3640,1804,653817.86,2061,185,168324.40,2539,286,1747,2347
01-03-2023,31-03-2023,2310,1237,4024,2742,1197285.16,1984,239,200716.69,3934,505,1685,2867
01-04-2023,30-04-2023,2487,638,4061,2439,969378.89,2099,254,113452.10,2705,369,1774,3565
01-05-2023,31-05-2023,3080,718,4424,3027,1758382.65,2217,359,168261.05,3128,654,1996,4618
01-06-2023,30-06-2023,2776,491,4304,2783,1054469.60,2385,403,184900.23,3152,443,1967,3939
01-07-2023,31-07-2023,3175,537,4890,3212,1573514.32,2669,498,149953.60,3589,596,2213,4540
01-08-2023,31-08-2023,4215,572,5428,3683,1767505.16,2993,575,227259.71,4096,721,2460,5407
01-09-2023,30-09-2023,3772,617,5870,3757,1371037.92,3290,642,215522.64,4661,588,2575,5280


In [19]:
grand_total = pivot_df.loc[('Total',), :]

In [20]:
df2 = pd.DataFrame(grand_total)

In [21]:
df2

,apc_wc_connected,bucket1contacted,bucket1normalized_denominator,bucket1normalized_greg,bucket1ptppmt,bucket2normalized_denominator,bucket2normalized_greg,bucket2ptppmt,bucketcallcontacted,kept_ptp,ptp_received,wccalled
WkendDate,,,,,,,,,,,,
,29903,8292,46560,28979,13080527.22,25615,4096,1874711.79,36881,5321,21354,40688


In [22]:
df

,WkstartDate,WkendDate,apc_wc_connected,wccalled,bucket1contacted,bucketcallcontacted,bucket1normalized_greg,bucket1normalized_denominator,bucket2normalized_denominator,bucket2normalized_greg,ptp_received,kept_ptp,bucket1ptppmt,bucket2ptppmt,FTE,total_accounts,active_works
0,01-01-2023,31-01-2023,2078,2458,1098,3485,1974,3613,2225,244,1917,327,764647.87,126419.52,23,18940,6608
1,01-02-2023,28-02-2023,2034,2347,842,2539,1804,3640,2061,185,1747,286,653817.86,168324.40,22,21398,6716
2,01-03-2023,31-03-2023,2310,2867,1237,3934,2742,4024,1984,239,1685,505,1197285.16,200716.69,21,23445,6921
3,01-04-2023,30-04-2023,2487,3565,638,2705,2439,4061,2099,254,1774,369,969378.89,113452.10,20,24623,6327
4,01-05-2023,31-05-2023,3080,4618,718,3128,3027,4424,2217,359,1996,654,1758382.65,168261.05,20,22604,6550
5,01-06-2023,30-06-2023,2776,3939,491,3152,2783,4304,2385,403,1967,443,1054469.60,184900.23,20,25791,6478
6,01-07-2023,31-07-2023,3175,4540,537,3589,3212,4890,2669,498,2213,596,1573514.32,149953.60,20,25434,6904
7,01-08-2023,31-08-2023,4215,5407,572,4096,3683,5428,2993,575,2460,721,1767505.16,227259.71,20,26972,7364
8,01-09-2023,30-09-2023,3772,5280,617,4661,3757,5870,3290,642,2575,588,1371037.92,215522.64,27,29436,8203
0,01-10-2023,30-10-2023,3976,5667,1542,5592,3558,6306,3692,697,3020,832,1970487.79,319901.85,27,33079,9133


In [23]:
df.FTE.iloc[-1]

27

In [24]:
df2['Welcomecallcallrate'] = (df2['apc_wc_connected']/df2['wccalled'])*100
df2['bucket1callcontactrate'] = (df2['bucket1contacted']/df2['bucketcallcontacted'])*100
df2['bucket1normalized'] = (df2['bucket1normalized_greg']/df2['bucket1normalized_denominator'])*100
df2['bucket2normalized'] = (df2['bucket2normalized_greg']/df2['bucket2normalized_denominator'])*100
df2['kept_ptp_percentage'] = (df2['kept_ptp']/df2['ptp_received'])*100
df2['totalptpmoneycollected'] = (df2['bucket1ptppmt'] + df2['bucket2ptppmt'])
df2['FTE'] = df.FTE.iloc[-1]
df2['Total_Accounts'] = df.total_accounts.iloc[-1]
df2['active_works'] = df.active_works.iloc[-1]
df2['Account_FTE'] = df['total_accounts'].iloc[-1]/df['FTE'].iloc[-1]
df2['accountsonactiveworksperfte'] = df['active_works'].iloc[-1]/df['FTE'].iloc[-1]

In [25]:
df2 = df2.reset_index()

In [26]:
df2

,WkendDate,apc_wc_connected,bucket1contacted,bucket1normalized_denominator,bucket1normalized_greg,bucket1ptppmt,bucket2normalized_denominator,bucket2normalized_greg,bucket2ptppmt,bucketcallcontacted,...,bucket1callcontactrate,bucket1normalized,bucket2normalized,kept_ptp_percentage,totalptpmoneycollected,FTE,Total_Accounts,active_works,Account_FTE,accountsonactiveworksperfte
0,,29903,8292,46560,28979,13080527.22,25615,4096,1874711.79,36881,...,22.483121,62.24012,15.99063,24.918048,14955239.01,27,33079,9133,1225.148148,338.259259


In [27]:
df2.drop(columns = ['WkendDate'], inplace = True)

In [28]:
df2['StartDate'] = '01-01-2023'
df2['EndDate'] = '31-10-2023'

In [29]:
df2.columns

Index(['apc_wc_connected', 'bucket1contacted', 'bucket1normalized_denominator',
       'bucket1normalized_greg', 'bucket1ptppmt',
       'bucket2normalized_denominator', 'bucket2normalized_greg',
       'bucket2ptppmt', 'bucketcallcontacted', 'kept_ptp', 'ptp_received',
       'wccalled', 'Welcomecallcallrate', 'bucket1callcontactrate',
       'bucket1normalized', 'bucket2normalized', 'kept_ptp_percentage',
       'totalptpmoneycollected', 'FTE', 'Total_Accounts', 'active_works',
       'Account_FTE', 'accountsonactiveworksperfte', 'StartDate', 'EndDate'],
      dtype='object')

In [30]:
df3 = df2[['StartDate', 'EndDate', 'apc_wc_connected', 'wccalled', 'Welcomecallcallrate', 'bucket1contacted', 'bucketcallcontacted', 'bucket1callcontactrate', 
           'bucket1normalized_greg', 'bucket1normalized_denominator','bucket1normalized', 'bucket2normalized_greg', 'bucket2normalized_denominator'
           , 'bucket2normalized', 'kept_ptp', 'ptp_received', 'kept_ptp_percentage', 'bucket1ptppmt', 'bucket2ptppmt','totalptpmoneycollected'
           ,  'FTE', 'Total_Accounts', 'active_works',  'Account_FTE', 'accountsonactiveworksperfte'
           ]].copy()

In [31]:
df3

,StartDate,EndDate,apc_wc_connected,wccalled,Welcomecallcallrate,bucket1contacted,bucketcallcontacted,bucket1callcontactrate,bucket1normalized_greg,bucket1normalized_denominator,...,ptp_received,kept_ptp_percentage,bucket1ptppmt,bucket2ptppmt,totalptpmoneycollected,FTE,Total_Accounts,active_works,Account_FTE,accountsonactiveworksperfte
0,01-01-2023,31-10-2023,29903,40688,73.493413,8292,36881,22.483121,28979,46560,...,21354,24.918048,13080527.22,1874711.79,14955239.01,27,33079,9133,1225.148148,338.259259


In [34]:
df3.to_excel("KPI_Till_October31st.xlsx", sheet_name="Jan12023 to Oct312023 Data", index = False)